# InfoGAN 

- Discrete category + continuous category
- LSGAN loss(https://arxiv.org/abs/1611.04076v2)

![alt text](./infogan.png)

## 1. Import required libraries

In [1]:
import torch
import torch.nn as nn
import torch.utils as utils
import torch.nn.init as init
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.utils as v_utils
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict

## 2. Hyperparameter & Data setting

In [2]:
# Set Hyperparameters

epoch = 50
batch_size = 128
learning_rate = 0.0002
num_gpus = 1
z_size= 62
discrete_latent_size = 10
contin_latent_size = 2
ratio = 0.75

# Download Data & Set Data Loader(input pipeline)

mnist_train = dset.MNIST("./", train=True, 
                         transform=transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                        ]),
                        target_transform=None,
                        download=True)
train_loader = torch.utils.data.DataLoader(dataset=mnist_train,batch_size=batch_size,shuffle=True,drop_last=True)

## 3. Label into one-hot vector

In [3]:
def int_to_onehot(z_label):
    one_hot_array = np.zeros(shape=[len(z_label), discrete_latent_size])
    one_hot_array[np.arange(len(z_label)), z_label] = 1
    return one_hot_array

## 4. Generator

In [4]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator,self).__init__()
        self.layer1 = nn.Sequential(
                    nn.Linear(z_size+discrete_latent_size+contin_latent_size,1024),
                    nn.BatchNorm1d(1024),
                    nn.ReLU(),               
                    nn.Linear(1024,7*7*256),               
                    nn.BatchNorm1d(7*7*256),
                    nn.ReLU(), 
            )              
        self.layer2 = nn.Sequential(OrderedDict([
                ('conv1', nn.ConvTranspose2d(256,128,3,2,1,1)), # [batch,256,7,7] -> [batch,128,14,14]
                ('bn1', nn.BatchNorm2d(128)),    
                ('relu1', nn.ReLU()),
                ('conv2', nn.ConvTranspose2d(128,64,3,1,1)),    # [batch,128,14,14] -> [batch,64,14,14]
                ('bn2', nn.BatchNorm2d(64)),    
                ('relu2', nn.ReLU()),
                
            ]))
        self.layer3 = nn.Sequential(OrderedDict([
                ('conv3',nn.ConvTranspose2d(64,32,3,1,1)),      # [batch,64,14,14] -> [batch,16,14,14]
                ('bn3',nn.BatchNorm2d(32)),    
                ('relu3',nn.ReLU()),
                ('conv4',nn.ConvTranspose2d(32,1,3,2,1,1)),     # [batch,16,14,14] -> [batch,1,28,28]
                ('tanh',nn.Tanh())
            ]))

    def forward(self,z):
        out = self.layer1(z)
        out = out.view(batch_size//num_gpus,256,7,7)
        out = self.layer2(out)
        out = self.layer3(out)
        return out


## 5. Discriminator

In [5]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator,self).__init__()
        self.layer1 = nn.Sequential(OrderedDict([
                ('conv1',nn.Conv2d(1,16,3,padding=1)),            # [batch,1,28,28] -> [batch,16,28,28]
                ('relu1',nn.LeakyReLU(0.1)),
                ('conv2',nn.Conv2d(16,32,3,stride=2,padding=1)),  # [batch,16,28,28] -> [batch,32,28,28]
                ('bn2',nn.BatchNorm2d(32)),    
                ('relu2',nn.LeakyReLU(0.1)),
                
            ]))
        self.layer2 = nn.Sequential(OrderedDict([
                ('conv3',nn.Conv2d(32,64,3,padding=1)),           # [batch,32,14,14] -> [batch,64,14,14] 
                ('relu3',nn.LeakyReLU(0.1)),
                ('bn3',nn.BatchNorm2d(64)),
                ('conv4',nn.Conv2d(64,128,3,stride=2,padding=1)), # [batch,64,7,7] -> [batch,128,7,7]
                ('bn4',nn.BatchNorm2d(128)),    
                ('relu4',nn.LeakyReLU(0.1))
            ]))
        self.fc = nn.Sequential(
                nn.Linear(128*7*7,1024),
                nn.BatchNorm1d(1024),
                nn.LeakyReLU(0.1),
                nn.Linear(1024,13),
                nn.Sigmoid()
            )
        '''
        self.fc2 = nn.Sequential(
                nn.Linear(128*7*7,10),
                nn.Sigmoid(),
            )
        self.fc3 = nn.Sequential(
                nn.Linear(128*7*7,2),
                nn.Sigmoid()
            )
        '''

    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(batch_size//num_gpus, -1)
        
        out = self.fc(out)
        
        output = out[:,0:1]
        onehot = out[:,1:11]
        contin = out[:,11:]
        
        return output,onehot,contin

## 6. Instance & Label on GPU

In [6]:
# put class instance on multi gpu

generator = nn.DataParallel(Generator(),device_ids=[0])
discriminator = nn.DataParallel(Discriminator(),device_ids=[0])

# put labels on multi gpu

ones_label = Variable(torch.ones(batch_size,1)).cuda()
zeros_label = Variable(torch.zeros(batch_size,1)).cuda()

## 7. Loss function & Optimizer

In [7]:
# this time, use LSGAN loss(https://arxiv.org/abs/1611.04076v2)

gan_loss_func = nn.BCELoss()
cat_loss_func = nn.CrossEntropyLoss()
contin_loss_func = nn.L1Loss()

gen_optim = torch.optim.Adam(generator.parameters(), lr= 5*learning_rate,betas=(0.5,0.999))
dis_optim = torch.optim.Adam(discriminator.parameters(), lr=learning_rate,betas=(0.5,0.999))

## 8. Model restore

In [8]:
try:
    generator, discriminator = torch.load('./model/infogan_catcon.pkl')
    print("\n--------model restored--------\n")
except:
    print("\n--------model not restored--------\n")
    pass


--------model not restored--------



In [ ]:
def contin_check():
    for i in range(10):
        z_random = np.random.rand(batch_size,z_size)
        z_onehot = np.random.randint(0, 10, size=batch_size)
        z_contin = np.random.uniform(-1,1,size=[batch_size,2])

        # change first 10 labels from random to 0~9          
        for l in range(40):
            z_onehot[l]=i

            if l <= 20: 
                z_contin[l,0]= (l-10)/3
                z_contin[l,1]= 0 
            else:      
                z_contin[l,0]= 0
                z_contin[l,1]= (l-30)/3

        #print(z_contin)

        # preprocess z
        z_label_onehot = int_to_onehot(z_onehot)
        z_concat = np.concatenate([z_random, z_label_onehot,z_contin], axis=1)
        z = Variable(torch.from_numpy(z_concat).type_as(torch.FloatTensor())).cuda()

        gen_fake = generator.forward(z)

        v_utils.save_image(gen_fake.data[0:40],"./result_contin/gen_{}.png".format(i),nrow=10)

## 9. Train Model

In [ ]:
for i in range(epoch):
    for j,(image,_) in enumerate(train_loader):
        
        # put image & label on gpu
        image = Variable(image).cuda()
    
        # discriminator
        
        z_random = np.random.normal(0,0.1,size=[batch_size,z_size])
        z_onehot = np.random.randint(0, 10, size=batch_size)
        z_contin = np.random.normal(0,1,size=[batch_size,2])

        # change first 10 labels from random to 0~9          
        for l in range(10):
            z_onehot[l]=l

        # preprocess z
        z_onehot_variable = Variable(torch.from_numpy(z_onehot)).cuda()
        z_label_onehot = int_to_onehot(z_onehot)
        z_concat = np.concatenate([z_random, z_label_onehot,z_contin], axis=1)

        z = Variable(torch.from_numpy(z_concat).type_as(torch.FloatTensor())).cuda()
        z_label_onehot = Variable(torch.from_numpy(z_label_onehot).type_as(torch.FloatTensor())).cuda()
        z_label_contin = Variable(torch.from_numpy(z_contin).type_as(torch.FloatTensor())).cuda()
        
        # dis_loss = gan_loss(fake & real) + categorical loss
        gen_fake = generator.forward(z)
        dis_fake, onehot_fake, contin_fake = discriminator.forward(gen_fake)
        
        dis_optim.zero_grad()
        dis_real, label_real, contin_real = discriminator.forward(image)
        dis_loss = torch.sum(gan_loss_func(dis_fake,zeros_label))\
                 + torch.sum(gan_loss_func(dis_real,ones_label))
                #\+ torch.sum(cat_loss_func(label_real,_))
                
        dis_loss.backward()
        dis_optim.step()
        
        
        # generator 
        #for k in range(2):
            
        z_random = np.random.normal(0,0.1,size=[batch_size,z_size])
        z_onehot = np.random.randint(0, 10, size=batch_size)
        z_contin = np.random.uniform(-1,1,size=[batch_size,2])

        # change first 10 labels from random to 0~9          
        for l in range(10):
            z_onehot[l]=l

        # preprocess z
        z_onehot_variable = Variable(torch.from_numpy(z_onehot)).cuda()
        z_label_onehot = int_to_onehot(z_onehot)
        z_concat = np.concatenate([z_random, z_label_onehot,z_contin], axis=1)

        z = Variable(torch.from_numpy(z_concat).type_as(torch.FloatTensor())).cuda()
        z_label_onehot = Variable(torch.from_numpy(z_label_onehot).type_as(torch.FloatTensor())).cuda()
        z_label_contin = Variable(torch.from_numpy(z_contin).type_as(torch.FloatTensor())).cuda()

        # calculate loss and apply gradients
        # gen_loss = gan loss(fake) + categorical loss
        gen_optim.zero_grad()
        gen_fake = generator.forward(z)
        dis_fake, onehot_fake, contin_fake = discriminator.forward(gen_fake)
        gen_loss = torch.sum(gan_loss_func(dis_fake,ones_label)) \
                 + ratio *(torch.sum(cat_loss_func(onehot_fake,z_onehot_variable))\
                 + torch.sum(contin_loss_func(contin_fake,z_label_contin)))

        gen_loss.backward()
        gen_optim.step()

        # model save
        if j % 10 == 0:
            torch.save([generator,discriminator],'./model/infogan_catcon.pkl')

            # print loss and image save
            print("{}th iteration gen_loss: {} dis_loss: {}".format(i,gen_loss.data,dis_loss.data))
            v_utils.save_image(gen_fake.data[0:20],"./result_catcon/gen_{}_{}.png".format(i,j), nrow=5)
    
    contin_check()

/home/gunho/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/serialization.py:147: UserWarning: Couldn't retrieve source code for container of type Generator. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/gunho/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/serialization.py:147: UserWarning: Couldn't retrieve source code for container of type Discriminator. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


0th iteration gen_loss: 
 2.9299
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 1.3977
[torch.cuda.FloatTensor of size 1 (GPU 0)]

0th iteration gen_loss: 
 2.6035
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 1.3566
[torch.cuda.FloatTensor of size 1 (GPU 0)]

0th iteration gen_loss: 
 2.6168
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 1.3132
[torch.cuda.FloatTensor of size 1 (GPU 0)]

0th iteration gen_loss: 
 2.6485
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 1.2733
[torch.cuda.FloatTensor of size 1 (GPU 0)]

0th iteration gen_loss: 
 2.7576
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 1.1973
[torch.cuda.FloatTensor of size 1 (GPU 0)]

0th iteration gen_loss: 
 2.9156
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 1.3253
[torch.cuda.FloatTensor of size 1 (GPU 0)]

0th iteration gen_loss: 
 2.9427
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 1.3422
[torch.cuda.FloatTensor of size 1 (GPU 0)]

0th iteration gen_lo

1th iteration gen_loss: 
 2.9963
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 1.2526
[torch.cuda.FloatTensor of size 1 (GPU 0)]

1th iteration gen_loss: 
 3.1456
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 1.1956
[torch.cuda.FloatTensor of size 1 (GPU 0)]

1th iteration gen_loss: 
 3.1026
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 1.2270
[torch.cuda.FloatTensor of size 1 (GPU 0)]

1th iteration gen_loss: 
 3.0628
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 1.1938
[torch.cuda.FloatTensor of size 1 (GPU 0)]

1th iteration gen_loss: 
 2.9556
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 1.2068
[torch.cuda.FloatTensor of size 1 (GPU 0)]

1th iteration gen_loss: 
 3.0782
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 1.1912
[torch.cuda.FloatTensor of size 1 (GPU 0)]

1th iteration gen_loss: 
 2.9782
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 1.1534
[torch.cuda.FloatTensor of size 1 (GPU 0)]

1th iteration gen_lo

2th iteration gen_loss: 
 3.0227
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 1.2135
[torch.cuda.FloatTensor of size 1 (GPU 0)]

2th iteration gen_loss: 
 3.3606
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 0.9801
[torch.cuda.FloatTensor of size 1 (GPU 0)]

2th iteration gen_loss: 
 3.3209
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 1.2188
[torch.cuda.FloatTensor of size 1 (GPU 0)]

2th iteration gen_loss: 
 3.1615
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 1.1849
[torch.cuda.FloatTensor of size 1 (GPU 0)]

2th iteration gen_loss: 
 3.1737
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 1.1138
[torch.cuda.FloatTensor of size 1 (GPU 0)]

2th iteration gen_loss: 
 3.1292
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 1.0319
[torch.cuda.FloatTensor of size 1 (GPU 0)]

2th iteration gen_loss: 
 3.4795
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 1.1203
[torch.cuda.FloatTensor of size 1 (GPU 0)]

2th iteration gen_lo

3th iteration gen_loss: 
 3.2183
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 0.9896
[torch.cuda.FloatTensor of size 1 (GPU 0)]

3th iteration gen_loss: 
 3.2345
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 1.0678
[torch.cuda.FloatTensor of size 1 (GPU 0)]

3th iteration gen_loss: 
 3.1807
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 1.0574
[torch.cuda.FloatTensor of size 1 (GPU 0)]

3th iteration gen_loss: 
 3.5768
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 1.0367
[torch.cuda.FloatTensor of size 1 (GPU 0)]

3th iteration gen_loss: 
 3.3962
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 1.0040
[torch.cuda.FloatTensor of size 1 (GPU 0)]

3th iteration gen_loss: 
 3.2741
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 0.9575
[torch.cuda.FloatTensor of size 1 (GPU 0)]

3th iteration gen_loss: 
 3.3781
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 1.0471
[torch.cuda.FloatTensor of size 1 (GPU 0)]

3th iteration gen_lo

...

7th iteration gen_loss: 
 3.7818
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 0.6743
[torch.cuda.FloatTensor of size 1 (GPU 0)]

7th iteration gen_loss: 
 3.5143
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 0.7367
[torch.cuda.FloatTensor of size 1 (GPU 0)]

7th iteration gen_loss: 
 4.1122
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 0.7688
[torch.cuda.FloatTensor of size 1 (GPU 0)]

7th iteration gen_loss: 
 3.7573
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 0.6472
[torch.cuda.FloatTensor of size 1 (GPU 0)]

7th iteration gen_loss: 
 3.8499
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 0.7347
[torch.cuda.FloatTensor of size 1 (GPU 0)]

7th iteration gen_loss: 
 3.7765
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 0.6414
[torch.cuda.FloatTensor of size 1 (GPU 0)]

7th iteration gen_loss: 
 3.8843
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 0.6913
[torch.cuda.FloatTensor of size 1 (GPU 0)]

7th iteration gen_lo

8th iteration gen_loss: 
 3.6849
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 0.7113
[torch.cuda.FloatTensor of size 1 (GPU 0)]

8th iteration gen_loss: 
 3.7640
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 0.4314
[torch.cuda.FloatTensor of size 1 (GPU 0)]

8th iteration gen_loss: 
 3.5099
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 0.5464
[torch.cuda.FloatTensor of size 1 (GPU 0)]

8th iteration gen_loss: 
 3.7520
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 0.7210
[torch.cuda.FloatTensor of size 1 (GPU 0)]

8th iteration gen_loss: 
 3.9827
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 0.6344
[torch.cuda.FloatTensor of size 1 (GPU 0)]

8th iteration gen_loss: 
 3.7084
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 0.4685
[torch.cuda.FloatTensor of size 1 (GPU 0)]

8th iteration gen_loss: 
 3.6496
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 0.7581
[torch.cuda.FloatTensor of size 1 (GPU 0)]

8th iteration gen_lo

10th iteration gen_loss: 
 3.6261
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 0.4907
[torch.cuda.FloatTensor of size 1 (GPU 0)]

10th iteration gen_loss: 
 4.0105
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 0.7370
[torch.cuda.FloatTensor of size 1 (GPU 0)]

10th iteration gen_loss: 
 3.5752
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 0.5387
[torch.cuda.FloatTensor of size 1 (GPU 0)]

10th iteration gen_loss: 
 3.4508
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 0.3432
[torch.cuda.FloatTensor of size 1 (GPU 0)]

10th iteration gen_loss: 
 3.9907
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 0.3793
[torch.cuda.FloatTensor of size 1 (GPU 0)]

10th iteration gen_loss: 
 3.7493
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 0.6794
[torch.cuda.FloatTensor of size 1 (GPU 0)]

10th iteration gen_loss: 
 3.7638
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 0.5049
[torch.cuda.FloatTensor of size 1 (GPU 0)]

10th iteratio

11th iteration gen_loss: 
 3.8353
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 0.4346
[torch.cuda.FloatTensor of size 1 (GPU 0)]

11th iteration gen_loss: 
 3.3637
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 0.2706
[torch.cuda.FloatTensor of size 1 (GPU 0)]

11th iteration gen_loss: 
 3.5481
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 0.3504
[torch.cuda.FloatTensor of size 1 (GPU 0)]

11th iteration gen_loss: 
 3.9027
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 0.3700
[torch.cuda.FloatTensor of size 1 (GPU 0)]

11th iteration gen_loss: 
 4.9336
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 0.2538
[torch.cuda.FloatTensor of size 1 (GPU 0)]

11th iteration gen_loss: 
 4.8121
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 0.4888
[torch.cuda.FloatTensor of size 1 (GPU 0)]

11th iteration gen_loss: 
 3.7217
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 0.4838
[torch.cuda.FloatTensor of size 1 (GPU 0)]

11th iteratio

12th iteration gen_loss: 
 3.7694
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 0.4195
[torch.cuda.FloatTensor of size 1 (GPU 0)]

12th iteration gen_loss: 
 3.7530
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 0.3662
[torch.cuda.FloatTensor of size 1 (GPU 0)]

12th iteration gen_loss: 
 3.8903
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 0.2643
[torch.cuda.FloatTensor of size 1 (GPU 0)]

12th iteration gen_loss: 
 3.7620
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 0.5126
[torch.cuda.FloatTensor of size 1 (GPU 0)]

12th iteration gen_loss: 
 4.1669
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 0.4585
[torch.cuda.FloatTensor of size 1 (GPU 0)]

12th iteration gen_loss: 
 3.8569
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 0.5015
[torch.cuda.FloatTensor of size 1 (GPU 0)]

12th iteration gen_loss: 
 4.3209
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 dis_loss: 
 0.4738
[torch.cuda.FloatTensor of size 1 (GPU 0)]

12th iteratio